In [2]:
### Run this to be able to access your drive. Google will ask you to input a verification code twice. Just do so

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp16ntlxyj/pubring.gpg' created
gpg: /tmp/tmp16ntlxyj/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [9]:
## Ensure your drive contains the folder ML Captcha at this bottom address. Run this to test if it really is there.

!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("/content/drive/ML Captcha")
!ls

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
dodyfolderdata.pickledody		     ML_model_test.h5
dodyfolderlabels.pickledody		     ML_model_test.pb
drive					     ML Train.ipynb
events.out.tfevents.1531566458.ab523be53b33  Modified_Claptcha_Lib.py
ML_Data_Quicksand			     __pycache__
ML_model.hdf5				     Quicksand-Regular.otf
ML_model_labels.dat			     Test_ML_Model.ipynb
ML_model_labels (temp).dat		     Visualise_Model.ipynb
ML_model (temp).hdf5


In [10]:
## Test Model

from keras.models import load_model
import os
import numpy as np
import cv2
import pickle
import os
from PIL import Image

# Number of tests to run
Number = 100

FOLDER = "/content/drive/ML Captcha/"

# Load the relevant files
MODEL_FILENAME = FOLDER + "ML_model.hdf5"
MODEL_LABELS_FILENAME = FOLDER + "ML_model_labels.dat"
CAPTCHA_IMAGE_FOLDER = FOLDER + "ML_Data_Quicksand"

# Open the labels file
with open(MODEL_LABELS_FILENAME, "rb") as f:
    lb = pickle.load(f)

# Load the model
model = load_model(MODEL_FILENAME)

# Get random captcha images from dataset
IMAGE_FILE_LIST = []

for folder in os.listdir(CAPTCHA_IMAGE_FOLDER):
  for file in os.listdir(CAPTCHA_IMAGE_FOLDER+"/"+folder):
    IMAGE_FILE_LIST += [str(CAPTCHA_IMAGE_FOLDER+"/"+folder+"/"+file)]
    
captcha_image_files = np.random.choice(IMAGE_FILE_LIST, size=(Number,), replace=False)

# Initialise counter
i = 0

# Loop over all the randomly chosen images
for image_file in captcha_image_files:
    
    # Load the image and convert it to grayscale
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Add 2 channel dimensions to the image to make Keras happy
    image = np.expand_dims(image, axis=2)
    image = np.expand_dims(image, axis=0)
    
    # Predict and convert prediction to an ordinary letter
    prediction = model.predict(image)
    prediction = lb.inverse_transform(prediction)[0]
    
    # Get folder of image to get the answer
    answer = image_file.split(os.path.sep)[-2]
    
    # Check if prediction correct, and print if prediction is wrong
    if (prediction == answer):
        # Increment counter
        i+=1
    else:
        
        # print wrong prediction
        print (prediction, answer)
        
# Print proportion of correct
print (i/Number)

Using TensorFlow backend.


1.0
